In [161]:
%reload_ext autoreload
%autoreload 2

In [162]:
import json
file = open('./../r_addresses.json')
wallet = json.load(file)

In [163]:
# lock p2tr script
from key import ECPubKey,ECKey
from messages import CTxIn,COutPoint,CTxOut,CTransaction,CTxWitness,CTxInWitness,ser_uint256
from script import tagged_hash, sha256,taproot_construct,SegwitV0SignatureHash, CScript, OP_1, OP_CHECKMULTISIG, OP_0,OP_CHECKSIG
import sys
sys.path.append("..")
from ecc import PrivateKey

prev_tx_hx = int("7c4e6957c0b39023414d60ec21a4a4194347d6d6f2353c6b1cabbcc284b71ef3", 16)
prv_amount=int(100*100000000)
prv_pub_key_hash = "7d7143a28a3b836e3043f3ba5dbcdc4ee29b7457"
prv_script=bytes.fromhex(f"76a914{prv_pub_key_hash}88ac")
prev_index = 1

p=ECKey()
p.set(0x00db5cdc1663eea25406126e642f72ef7c78f68d249589d2873c613e61929eea)

target_public_key_bytes=bytes.fromhex(wallet[12]["public key"])
target_amount = int(99.999*100000000)

tx_in =CTxIn(COutPoint(prev_tx_hx, prev_index))

target_public_key=ECPubKey().set(target_public_key_bytes)
mast_scripts = [ CScript([target_public_key.get_bytes(), OP_CHECKSIG]) ]

locking_script, tweak, control_map = taproot_construct(target_public_key, mast_scripts)

tx_outs = CTxOut(target_amount, locking_script)

tx = CTransaction()
tx.vin=[tx_in]
tx.vout=[tx_outs]
tx.calc_sha256()
tx_sig_hash = SegwitV0SignatureHash(prv_script, tx, 0, 0x01, prv_amount)
tx_input_0_wit = CTxInWitness([p.sign_ecdsa(tx_sig_hash) + (1).to_bytes(1, 'big'), p.get_pubkey().get_bytes(False)])
tx.wit.vtxinwit = [tx_input_0_wit]

print(tx.serialize_with_witness().hex())

# Use for Next Transaction
tx1_outs = tx.vout
t = tweak
cmp = control_map


01000000000101f31eb784c2bcab1c6b3c35f2d6d6474319a4a421ec604d412390b3c057694e7c0100000000ffffffff01605d0a5402000000225120d53a7554fa8af2453ed8ca4c88c227f55019b771f103c2bf9d2c2e9b341a81ce02483045022100ddb91e78adf0c0f4e18f5ddfeb6b8ff9499dc70a6fd88dd3d6589bba318a0a87022026eed118efd54fb7bffd1414da0db3810ac79341b5ecb93ab4e531a2e4db08bc012103eb477934368b96afb4e7878b96f29bc4aec8238189652f17a7b33eb0ed6acbee00000000


In [164]:
# unlocking p2tr script path
from messages import ser_string
from script import get_p2wpkh_script, TaprootSignatureHash
from segwit_addr import decode_segwit_address
from key import SECP256K1_ORDER, SECP256K1, SECP256K1_G, int_or_bytes

taproot_locking_tx_id = int("6ca1cb6a524c195b81fa48f0f84f06f203f49e47ce62de9e718d6e7c4391f9db", 16)
taproot_locking_tx_index = 0

taproot_unlocking_sec_key = ECKey().set(bytes.fromhex("13e4f9204068a232f23fe6305bb6c1fcb6d2c6e53347956a7fbaf2953737ab4c"))

target_amount = int(99.998*100000000)
target_b32_address = wallet[0]["address"]

tx_in_0 = CTxIn(COutPoint(taproot_locking_tx_id, taproot_locking_tx_index))
tx_ins = [tx_in_0]

locking_public_key_hash = bytes(decode_segwit_address("bcrt", target_b32_address)[1])
locking_script = get_p2wpkh_script(locking_public_key_hash)
tx_out_0 = CTxOut(target_amount, locking_script)
tx_outs = [tx_out_0]

tx = CTransaction()
tx.vin = tx_ins
tx.vout = tx_outs
tx.calc_sha256()

prev_tx_outs = tx1_outs
tx_sig_hash_0 = TaprootSignatureHash(tx, prev_tx_outs, 0X00, 0, True, list(cmp)[0])
taproot_unlocking_sec_key.negate()
tx_sign_0 = taproot_unlocking_sec_key.sign_schnorr(tx_sig_hash_0)

tx_input_0_wit = CTxInWitness([ tx_sign_0, list(cmp)[0], cmp[list(cmp)[0]]])
tx.wit.vtxinwit = [tx_input_0_wit]
print(tx.serialize_with_witness().hex())



01000000000101dbf991437c6e8d719ede62ce479ef403f2064ff8f048fa815b194c526acba16c0000000000ffffffff01c0d60854020000001600147d7143a28a3b836e3043f3ba5dbcdc4ee29b74570340d3f1c9df6b46cc7377eff03f0d6414137b69245d81fed3904acb252048bbd99183d305751f8f0751dfaaf29ba511b03d55dfaf19671d6cd3b84653fb8123704d2220728f0593227b1ba7d31914bf2f83d7335e12fdb92c7b39d12999594b61997a19ac21c0728f0593227b1ba7d31914bf2f83d7335e12fdb92c7b39d12999594b61997a1900000000


In [40]:
from script import taproot_tree_helper
internal_pub_key = ECPubKey().set(bytes.fromhex("187791b6f712a8ea41c8ecdd0ee77fab3e85263b37e1ec18a3651926b3a6cf27"))
locking_script, tweak, cmp = taproot_construct(internal_pub_key, [ (0xc0, CScript(bytes.fromhex("20d85a959b0290bf19bb89ed43c916be835475d013da4b362117393e25a48229b8ac")) )])

print(taproot_tree_helper([ (0xc0, CScript(bytes.fromhex("20d85a959b0290bf19bb89ed43c916be835475d013da4b362117393e25a48229b8ac")) )])[1].hex())
print(internal_pub_key.tweak_add(tweak).get_bytes().hex())
print(locking_script.hex())
print(tweak.hex())
print(cmp[list(cmp)[0]].hex())



5b75adecf53548f3ec6ad7d78383bf84cc57b55a3127c72b9a2481752dd88b21
147c9c57132f6e7ecddba9800bb0c4449251c92a1e60371ee77557b6620f3ea3
5120147c9c57132f6e7ecddba9800bb0c4449251c92a1e60371ee77557b6620f3ea3
cbd8679ba636c1110ea247542cfbd964131a6be84f873f7f3b62a777528ed001
c1187791b6f712a8ea41c8ecdd0ee77fab3e85263b37e1ec18a3651926b3a6cf27


In [120]:
bye = b'\t\\\xd0\xa9XTK\x88\xd7\xacX\xa2\xf2\x9c\xcdI\xc9VZ\x9f\x7f\xe7\xaa\x96m\xf4\x19\xbf}\xa0\xa5+'
print(bye.hex())

bye2 = b'1\x96\x9f\xa4\x08\xe5\x1b/\xec1!\xebjM*\xc8\x02\xa9\x87\xaa\x89\nHdw\xb72\x0e\xb5\x107z'
print(bye2.hex())

print(SECP256K1.lift_x(int.from_bytes(pub_key, 'big')))

print("43273777186443610501550240681962765976649068965092868303218065941808905780180")

095cd0a958544b88d7ac58a2f29ccd49c9565a9f7fe7aa966df419bf7da0a52b
31969fa408e51b2fec3121eb6a4d2ac802a987aa890a486477b7320eb510377a
(51816362344230832880300216262509008239902364748236324249889473786728335112729, 4162054245958954304649435371766049053427989041172356308662552853458952477290, 1)
51816362344230832880300216262509008239902364748236324249889473786728335112729
